<h2>Benifits of doing preprocessing inside the model at inference time</h2>
<p>If we choose option-2 (explained in above section), we may later want to export an inference-only end to end model that will include the preprocessing layers. The key benifit of doing this is that it makes our model portable and it helps to reduce the training or serving screw. <br>
When all data preprocessing is part of the model, other people can load and use the model without having to be aware of how each feature is expected to be encoded and normalized. The inference model will be able to process raw images or raw structured data and will not require users aware of the details like tokenization scheme used for the text, indexing scheme used for categorical values etc. This is especially powerful if we're exporting the model to another runtime such as TensorFLow.js. We won't have to again implement the preprocessing pipeline in javascript.<br>
If we initially put the preprocessing layers in our tf.data pipeline, we can export an inference model that packages the preprocessing. Simply instantiate that chains our preprocessing layer and our training model.</p>


```
#Creating input and preprocessing layer
inputs = keras.Input(shape=input_shape)
x = preprocessing_layer(inputs)

#creating output layer
outputs = training_model(x)

# Creating inference model with inputs and outputs
inference_model = keras.Model(inputs, outputs)
```



<h2>Quick recipes</h2>
<h4>Image data augmentation (on-device)</h4>
<p>It is important to note that, image data augmentation layers are only active during training. This layer is similar to the Dropout layer.</p>


In [10]:
# Importing keras and keras layers
import numpy as np
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

# Create a data augmentation stage with horizontal flipping, rotations, zooms
data_augmentation = keras.Sequential(
    [
        preprocessing.RandomFlip("horizontal"),
        preprocessing.RandomRotation(0.1),
        preprocessing.RandomZoom(0.1),
    ]
)

# Create a model that includes the augmentation stage
input_shape = (32, 32, 3)
classes = 10
inputs = keras.Input(shape=input_shape)

# Create augment images
x = data_augmentation(inputs)

# Rescale image values to [0, 1]
x = preprocessing.Rescaling(1.0 / 255)(x)

# Add the rest of the model
outputs = keras.applications.ResNet50(
    weights=None, input_shape=input_shape, classes=classes
)(x)

# Create the keras model with inputs and outputs
model = keras.Model(inputs, outputs)

<h4>Normalizing numerical features</h4>

In [11]:
# Load cifar10 dataset of keras
(x_train, y_train), _ = keras.datasets.cifar10.load_data()

# preprocessing the data
x_train = x_train.reshape((len(x_train), -1))
input_shape = x_train.shape[1:]
classes = 10

# Create a Normalization layer and set its internal state using the training data
normalizer = preprocessing.Normalization()
normalizer.adapt(x_train)

# Create a model that include the normalization layer 
inputs = keras.Input(shape=input_shape)
x = normalizer(inputs)
outputs = layers.Dense(classes, activation="softmax")(x)

#Creating the model
model = keras.Model(inputs, outputs)

# Train the model with Adam optimizer
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy")
model.fit(x_train, y_train)

1563/1563 [==============================] - 4s 2ms/step - loss: 2.1233


<h4>Encoding string categorical features via one-hot encoding</h4>


In [12]:
# Create some toy data using tf.constant
data = tf.constant([["a"], ["b"], ["c"], ["b"], ["c"], ["a"]])

# Use StringLookup to build an index of the feature values and encode output.
lookup = preprocessing.StringLookup(output_mode="binary")
lookup.adapt(data)

# Convert new test data (which includes unknown feature values)
test_data = tf.constant([["a"], ["b"], ["c"], ["d"], ["e"], [""]])

# Encode the test data using lookup() function and display it 
encoded_data = lookup(test_data)
print(encoded_data)

tf.Tensor(
[[0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 0.]], shape=(6, 4), dtype=float32)


<p>In the above output, 0 is reserved for missing values (which we should specify as the empty string "") and index 1 is reserved for out of vocabulary values (values that are not seen during adapt()). We can configure this by using the mask_token and oov_token constructor arguements of StringLookup.</p>


<h4>Encoding integer categorical features via one-hot encoding</h4>

In [13]:
# Create some toy data using tf.constant
data = tf.constant([[10], [20], [20], [10], [30], [0]])

# Use IntegerLookup to build an index of the feature values and encode output.
lookup = preprocessing.IntegerLookup(output_mode="binary")
lookup.adapt(data)

# Convert new test data (which includes unknown feature values)
test_data = tf.constant([[10], [10], [20], [50], [60], [0]])

#Encode the test data using lookup function and display it
encoded_data = lookup(test_data)
print(encoded_data)

tf.Tensor(
[[0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 0.]], shape=(6, 4), dtype=float32)


<p>In the above output, index 0 is reserved for missing values (which we specify as the value 0), and index 1 is reserved for out-of-vocabulary values (which are values not seen during adapt() method).We can configure this by using the mask_token and oov_token constructor arguements of IntegerLookup.</p>

<h4>Applying the hashing trick to an integer categorical feature</h4>
<p>If we have a categorical feature that can take many different values (on the order of 10e3 or higher), where each value only appears a few times in the data, it becomes impractical and ineffective to index one-hot encode the feature values. SUppose doing all these, it can be good idea to apply the <b>hashing trick</b>, in which hash the values to a vector of fixed size. This keeps the size of the feature space manageable and removes the need for explicit indexing.</p> 

In [14]:
# creating data of 10,000 random integers with values between 0 and 100,000
data = np.random.randint(0, 100000, size=(10000, 1))

# Use the Hashing layer to hash the values to the range [0, 64]
hasher = preprocessing.Hashing(num_bins=64, salt=1337)

# Use the CategoryEncoding layer to one-hot encode the hashed values
encoder = preprocessing.CategoryEncoding(num_tokens=64, output_mode="binary")
encoded_data = encoder(hasher(data))

#Displaying the encoded data
print(encoded_data.shape)

(10000, 64)


<h4>Encoding text as a sequence of token indices</h4>
<p>This is the way we preprocess text to be passed to an embedding layer.</p>

In [15]:
# Create the data using tf.constant
data = tf.constant(
    [
        "The Brain is wider than the Sky",
        "For put them side by side",
        "The one the other will contain",
        "With ease and You beside",
    ]
)
# Instantiate TextVectorization with int output_mode
text_vectorizer = preprocessing.TextVectorization(output_mode="int")

# Index the vocabulary via adapt() method
text_vectorizer.adapt(data)

# You can retrieve the vocabulary we indexed via get_vocabulary()
vocab = text_vectorizer.get_vocabulary()
print("Vocabulary:", vocab)

# Create an Embedding and LSTM layer
inputs = keras.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = layers.Embedding(input_dim=len(vocab), output_dim=64)(x)
outputs = layers.LSTM(1)(x)

#Creating a model
model = keras.Model(inputs, outputs)

# Call the model on test data (which includes unknown tokens)
test_data = tf.constant(["The Brain is deeper than the sea"])
test_output = model(test_data)

Vocabulary: ['', '[UNK]', 'the', 'side', 'you', 'with', 'will', 'wider', 'them', 'than', 'sky', 'put', 'other', 'one', 'is', 'for', 'ease', 'contain', 'by', 'brain', 'beside', 'and']


<p> When training model like above, for the best performance, we should use TextVectorization layer as part of the input pipeline.</p>

<h4>Encoding text as a dense matrix of ngrams with multi-hot encoding</h4>
<p>This shows the way we should preprocess text to be passed to Dense layer.</p> 

In [16]:
# Create text data using tf.constant
data = tf.constant(
    [
        "The Brain is wider than the Sky",
        "For put them side by side",
        "The one the other will contain",
        "With ease and You beside",
    ]
)
# Instantiate TextVectorization with "binary" output_mode (multi-hot)
# and ngrams=2 (index all bigrams)
text_vectorizer = preprocessing.TextVectorization(output_mode="binary", ngrams=2)

# Index the bigrams via adapt() method
text_vectorizer.adapt(data)

print(
    "Encoded text:\n",
    text_vectorizer(["The Brain is deeper than the sea"]).numpy(),
    "\n",
)

# Create a text_vectorize and Dense layer
inputs = keras.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
outputs = layers.Dense(1)(x)

#Creating the model
model = keras.Model(inputs, outputs)

# Call the model on test data (which includes unknown tokens)
test_data = tf.constant(["The Brain is deeper than the sea"])
test_output = model(test_data)

#Displaying model output
print("Model output:", test_output)

Encoded text:
 [[1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0.]] 

Model output: tf.Tensor([[0.6366095]], shape=(1, 1), dtype=float32)


<h4>Encoding text as a dense matrix of ngrams with TF-IDF weighting</h4>
<p>This is an alternative way of preprocessing text before passing it to a Dense layer.</p>

In [17]:
# Create text data using tf.constant
data = tf.constant(
    [
        "The Brain is wider than the Sky",
        "For put them side by side",
        "The one the other will contain",
        "With ease and You beside",
    ]
)
# Instantiate TextVectorization with tf-idf output_mode
# (multi-hot with TF-IDF weighting) and ngrams=2 (index all bigrams)
text_vectorizer = preprocessing.TextVectorization(output_mode="tf-idf", ngrams=2)

# Index the bigrams and learn the TF-IDF weights via adapt() method
text_vectorizer.adapt(data)

print(
    "Encoded text:\n",
    text_vectorizer(["The Brain is deeper than the sea"]).numpy(),
    "\n",
)

# Create text_vectorize and Dense layer
inputs = keras.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
outputs = layers.Dense(1)(x)

#Create the model
model = keras.Model(inputs, outputs)

# Call the model on test data (which includes unknown tokens)
test_data = tf.constant(["The Brain is deeper than the sea"])
test_output = model(test_data)

#Display the model output
print("Model output:", test_output)

Encoded text:
 [[5.461647  1.6945957 0.        0.        0.        0.        0.
  0.        0.        0.        0.        0.        0.        0.
  0.        0.        1.0986123 1.0986123 1.0986123 0.        0.
  0.        0.        0.        0.        0.        0.        0.
  1.0986123 0.        0.        0.        0.        0.        0.
  0.        1.0986123 1.0986123 0.        0.        0.       ]] 

Model output: tf.Tensor([[-2.9154975]], shape=(1, 1), dtype=float32)
